In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#sns.set() # to change theme

#from google.colab import drive
#drive.mount('/content/drive')
#display only 1 digit after decimal point
#pd.options.display.float_format = '{:.1f}'.format

In [ ]:
activity = pd.read_excel('/Users/raphael/Documents/STENDO-activities/activity.xlsx')

# Travail en arrière plan

In [ ]:
print('activity shape =', activity.shape)

In [ ]:
#activity

In [ ]:
activity.isna().sum()

In [ ]:
#activity[activity['percentage_real_duration'].isna()]

In [ ]:
activity['percentage_real_duration'] = activity['real_duration'] / (activity['duration'] * 60) * 100

In [ ]:
#activity

In [ ]:
#activity.isna().sum()

In [ ]:
activity = activity.drop(columns=['mac_address', 'name', 'care_note', 'credit_number', 'active'])

In [ ]:
#activity

In [ ]:
activity_europe = activity[(activity['ps_server'] == 'EU') & (activity['spare_type'] == 'pants')]
#activity_europe

In [ ]:
activity_europe = activity_europe[activity_europe['full_name.1'] != 'T0362 - LETESTEUR Ludovic']
#activity_europe

In [ ]:
activity_asia = activity[(activity['ps_server'] == 'ASIA') & (activity['spare_type'] == 'pants')]
#activity_asia

In [ ]:
activity_europe_50 = activity_europe[activity_europe['percentage_real_duration'] >= 50]
#activity_europe_50

In [ ]:
activity_asia_50 = activity_asia[activity_asia['percentage_real_duration'] >= 50]
#activity_asia_50

In [ ]:
activity_europe_50 = activity_europe_50.sort_values(by=['full_name.1'])
#activity_europe_50

In [ ]:
sorted_activity_europe_50 = activity_europe_50.groupby('full_name.1').size().reset_index(name='counts')
#sorted_activity_europe_50

In [ ]:
first_activity_dates = activity_europe_50.groupby('full_name.1')['start_date'].min().reset_index()
first_activity_dates.rename(columns={'start_date': 'first_activity'}, inplace=True)
sorted_activity_europe_50 = pd.merge(sorted_activity_europe_50, first_activity_dates, on='full_name.1', how='left')
#sorted_activity_europe_50

In [ ]:
last_activity_dates = activity_europe_50.groupby('full_name.1')['start_date'].max().reset_index()
last_activity_dates.rename(columns={'start_date': 'last_activity'}, inplace=True)
sorted_activity_europe_50 = pd.merge(sorted_activity_europe_50, last_activity_dates, on='full_name.1', how='left')
#sorted_activity_europe_50

In [ ]:
import datetime

sorted_activity_europe_50['first_activity'] = pd.to_datetime(sorted_activity_europe_50['first_activity'])
sorted_activity_europe_50['last_activity'] = pd.to_datetime(sorted_activity_europe_50['last_activity'])
# changer le mode de calcul du nombre de jours. Ce n'est plus le nolbre de jours entre la première et la dernière activité mais le nombre de jours entre la première activité et la date du jour
# sorted_activity_europe_50['last_activity'] = pd.to_datetime(sorted_activity_europe_50['last_activity'])
sorted_activity_europe_50['number_days'] = 1 + (pd.to_datetime(datetime.date.today()) - sorted_activity_europe_50['first_activity']).dt.days
#sorted_activity_europe_50

In [ ]:
# prompt: Add an extra column to sorted_activity_europe_50 called activity_last_7_days. The entries of the column activity_last_7_days should be equal to the counts of entries of full_name.1 between the current date and the curret date minus7 days

# Assuming 'start_date' column is already datetime objects. If not, convert it first:
# sorted_activity_europe_50['start_date'] = pd.to_datetime(sorted_activity_europe_50['start_date'])

import datetime

# Ensure 'start_date' in activity_europe_50 is datetime
activity_europe_50['start_date'] = pd.to_datetime(activity_europe_50['start_date'])

def count_activity_last_n_days(df, user, n_days):
    today = datetime.date.today()
    start_date = today - datetime.timedelta(days=n_days)
    user_activity = df[df['full_name.1'] == user]
    last_n_days_activity = user_activity[
        (user_activity['start_date'].dt.date >= start_date) &
        (user_activity['start_date'].dt.date <= today)
    ]
    return len(last_n_days_activity)

activity_data = []
for user in sorted_activity_europe_50['full_name.1']:
    activity_data.append({
        'full_name.1': user,
        'activity_last_7_days': count_activity_last_n_days(activity_europe_50, user, 7),
        'activity_last_14_days': count_activity_last_n_days(activity_europe_50, user, 14),
         'activity_last_30_days': count_activity_last_n_days(activity_europe_50, user, 30),
         'activity_last_90_days': count_activity_last_n_days(activity_europe_50, user, 90),
         'activity_last_365_days': count_activity_last_n_days(activity_europe_50, user, 365)
    })

activity_df = pd.DataFrame(activity_data)
#merged the two dataframes  activity_df and sorted_activity_europe_50
sorted_activity_europe_50 = pd.merge(sorted_activity_europe_50, activity_df, on='full_name.1', how='left')

In [ ]:
sorted_activity_europe_50['activity_per_day_since_acquisition'] = sorted_activity_europe_50['counts'] / sorted_activity_europe_50['number_days']
sorted_activity_europe_50['activity_per_day_7_days'] = sorted_activity_europe_50['activity_last_7_days'] / 5
sorted_activity_europe_50['activity_per_day_14_days'] = sorted_activity_europe_50['activity_last_14_days'] / 10
sorted_activity_europe_50['activity_per_day_30_days'] = sorted_activity_europe_50['activity_last_30_days'] / 20
sorted_activity_europe_50['activity_per_day_90_days'] = sorted_activity_europe_50['activity_last_90_days'] / 55
sorted_activity_europe_50['activity_per_day_year'] = sorted_activity_europe_50['activity_last_365_days'] / 220
#sorted_activity_europe_50
#round the result to 1 digit after the comma
sorted_activity_europe_50 = sorted_activity_europe_50.round({'activity_per_day_since_acquisition': 2, 'activity_per_day_7_days': 2, 'activity_per_day_14_days': 2, 'activity_per_day_30_days': 2, 'activity_per_day_90_days': 2, 'activity_per_day_year': 2})

In [ ]:
#sorted_activity_europe_50 = sorted_activity_europe_50.sort_values(by=['activity_per_day_since_acquisition'], ascending=False)

#Test
sorted_activity_europe_50 = sorted_activity_europe_50.sort_values(by=['activity_last_365_days'], ascending=False)
#fin test


sorted_activity_europe_50

In [ ]:
#supression du calcul du rang des activités totales

#sorted_activity_europe_50['rank_activity_per_day_7_days'] = sorted_activity_europe_50['activity_last_7_days'].rank(ascending=False, method='min').astype(int)
#sorted_activity_europe_50['rank_activity_per_day_14_days'] = sorted_activity_europe_50['activity_last_14_days'].rank(ascending=False, method='min').astype(int)
#sorted_activity_europe_50['rank_activity_per_day_30_days'] = sorted_activity_europe_50['activity_last_30_days'].rank(ascending=False, method='min').astype(int)
#sorted_activity_europe_50['rank_activity_per_day_90_days'] = sorted_activity_europe_50['activity_last_90_days'].rank(ascending=False, method='min').astype(int)
#sorted_activity_europe_50['rank_activity_per_day_year'] = sorted_activity_europe_50['activity_last_365_days'].rank(ascending=False, method='min').astype(int)
sorted_activity_europe_50['rank_activity_per_day_since_acquisition'] = sorted_activity_europe_50['activity_per_day_since_acquisition'].rank(ascending=False, method='min').astype(int)
sorted_activity_europe_50['rank_activity_per_day_7_days'] = sorted_activity_europe_50['activity_per_day_7_days'].rank(ascending=False, method='min').astype(int)
sorted_activity_europe_50['rank_activity_per_day_14_days'] = sorted_activity_europe_50['activity_per_day_14_days'].rank(ascending=False, method='min').astype(int)
sorted_activity_europe_50['rank_activity_per_day_30_days'] = sorted_activity_europe_50['activity_per_day_30_days'].rank(ascending=False, method='min').astype(int)
sorted_activity_europe_50['rank_activity_per_day_90_days'] = sorted_activity_europe_50['activity_per_day_90_days'].rank(ascending=False, method='min').astype(int)
sorted_activity_europe_50['rank_activity_per_day_year'] = sorted_activity_europe_50['activity_per_day_year'].rank(ascending=False, method='min').astype(int)

#sorted_activity_europe_50

In [ ]:
sorted_activity_europe_50['percentage_total_activity_per_day'] = (sorted_activity_europe_50['activity_per_day_since_acquisition'] / sorted_activity_europe_50['activity_per_day_since_acquisition'].sum()) * 100
#sorted_activity_europe_50

In [ ]:
sorted_activity_europe_50['cumulated_percentage_total_activity_per_day'] = sorted_activity_europe_50['percentage_total_activity_per_day'].cumsum()
#sorted_activity_europe_50

In [ ]:


from datetime import datetime
from dateutil import relativedelta
import calendar



#add a column to sorted_activity_europe_50 called today, it should be equal to the current date
sorted_activity_europe_50['today'] = datetime.today().date()

# Function to format timedelta using relativedelta
def format_timedelta(start_date, end_date):
    delta = relativedelta.relativedelta(end_date, start_date)
    years = delta.years
    months = delta.months
    days = delta.days
    return f"{years} an(s), {months} mois, {days} jour(s)"

# Example usage with your DataFrame:
sorted_activity_europe_50['formatted_duration'] = sorted_activity_europe_50.apply(
    lambda row: format_timedelta(row['first_activity'], row['today']), axis=1
)

# Liste des clients non rentables

# Sur 7 jours

In [ ]:
# @title
# Display sorted_activity_europe_50 with activity_last_7_days < 1.25, showing only 'full_name.1' and 'activity_last_7_days'
display_unprofitable_clients_activity_per_days_7_days = sorted_activity_europe_50[sorted_activity_europe_50['activity_per_day_7_days'] < 1.25][['full_name.1', 'activity_per_day_7_days']].sort_values(by=['activity_per_day_7_days'], ascending=False)
display_unprofitable_clients_activity_per_days_7_days = display_unprofitable_clients_activity_per_days_7_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_7_days': 'Nombre moyen séance par jours sur les 7 derniers jours',
        })
print(f"{len(display_unprofitable_clients_activity_per_days_7_days)} clients sur "+ f"{len(sorted_activity_europe_50)}" + " ne sont pas rentables sur les 7 derniers jours")
display(display_unprofitable_clients_activity_per_days_7_days.style.format({"Nombre moyen séance par jours sur les 7 derniers jours": "{:.2f}"}).hide(axis='index'))

# Sur  14 jours

In [ ]:
# @title
display_unprofitable_clients_activity_per_days_14_days = sorted_activity_europe_50[sorted_activity_europe_50['activity_per_day_14_days'] < 1.25][['full_name.1', 'activity_per_day_14_days']].sort_values(by=['activity_per_day_14_days'], ascending=False)
display_unprofitable_clients_activity_per_days_14_days = display_unprofitable_clients_activity_per_days_14_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_14_days': 'Nombre moyen séance par jours sur les 14 derniers jours',
        })
print(f"{len(display_unprofitable_clients_activity_per_days_14_days)} clients sur "+ f"{len(sorted_activity_europe_50)}" + " ne sont pas rentables sur les 14 derniers jours")
display(display_unprofitable_clients_activity_per_days_14_days.style.format({"Nombre moyen séance par jours sur les 14 derniers jours": "{:.2f}"}).hide(axis='index'))

# Sur 30 jours

In [ ]:
# @title
display_unprofitable_clients_activity_per_days_30_days = sorted_activity_europe_50[(sorted_activity_europe_50['activity_per_day_30_days'] < 1.25) & (sorted_activity_europe_50['number_days'] > 30)][['full_name.1', 'activity_per_day_30_days']].sort_values(by=['activity_per_day_30_days'], ascending=False)
display_unprofitable_clients_activity_per_days_30_days = display_unprofitable_clients_activity_per_days_30_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_30_days': 'Nombre moyen séance par jours sur les 30 derniers jours',
        })
print(f"{len(display_unprofitable_clients_activity_per_days_30_days)} clients sur "+ f"{len(sorted_activity_europe_50[sorted_activity_europe_50['number_days']>30])}" + " ne sont pas rentables sur les 30 derniers jours")
display(display_unprofitable_clients_activity_per_days_30_days.style.format({"Nombre moyen séance par jours sur les 30 derniers jours": "{:.2f}"}).hide(axis='index'))

# Sur 90 jours

In [ ]:
# @title
display_unprofitable_clients_activity_per_days_90_days = sorted_activity_europe_50[(sorted_activity_europe_50['activity_per_day_90_days'] < 1.25) & (sorted_activity_europe_50['number_days'] > 90)][['full_name.1', 'activity_per_day_90_days']].sort_values(by=['activity_per_day_90_days'], ascending=False)
display_unprofitable_clients_activity_per_days_90_days = display_unprofitable_clients_activity_per_days_90_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_90_days': 'Nombre moyen séance par jours sur les 90 derniers jours',
        })
print(f"{len(display_unprofitable_clients_activity_per_days_90_days)} clients sur "+ f"{len(sorted_activity_europe_50[sorted_activity_europe_50['number_days']>90])}" + " ne sont pas rentables sur les 90 derniers jours")
display(display_unprofitable_clients_activity_per_days_90_days.style.format({"Nombre moyen séance par jours sur les 90 derniers jours": "{:.2f}"}).hide(axis='index'))

# Sur 1 an

In [ ]:
# @title
display_unprofitable_clients_activity_per_days_year = sorted_activity_europe_50[(sorted_activity_europe_50['activity_per_day_year'] < 1.25) & (sorted_activity_europe_50['number_days'] > 365)][['full_name.1', 'activity_per_day_year']].sort_values(by=['activity_per_day_year'], ascending=False)
display_unprofitable_clients_activity_per_days_year = display_unprofitable_clients_activity_per_days_year.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_year': 'Nombre moyen séance par jours sur 1 an',
        })
print(f"{len(display_unprofitable_clients_activity_per_days_year)} clients sur "+ f"{len(sorted_activity_europe_50[sorted_activity_europe_50['number_days']>365])}" + " ne sont pas rentables sur 1 an")
display(display_unprofitable_clients_activity_per_days_year.style.format({"Nombre moyen séance par jours sur 1 an": "{:.2f}"}).hide(axis='index'))

# Depuis la première utilisation

In [ ]:
# @title
display_unprofitable_clients_activity_per_days_year = sorted_activity_europe_50[sorted_activity_europe_50['activity_per_day_since_acquisition'] < 0.87][['full_name.1', 'activity_per_day_since_acquisition']].sort_values(by=['activity_per_day_since_acquisition'], ascending=False)
display_unprofitable_clients_activity_per_days_year = display_unprofitable_clients_activity_per_days_year.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_since_acquisition': 'Nombre moyen séance par jours sur 1 an',
        })
print(f"{len(display_unprofitable_clients_activity_per_days_year)} clients sur "+ f"{len(sorted_activity_europe_50)}" + " ne sont pas rentables")
display(display_unprofitable_clients_activity_per_days_year.style.format({"Nombre moyen séance par jours depuis la première séance": "{:.2f}"}).hide(axis='index'))

# Classement des 30 premiers clients européens en fonction du nombre moyen de séances par jour


# Depuis l'acquisition de l'appareil V4

In [ ]:
# @title
display_30_best_clients_activity_per_days_entire_period = sorted_activity_europe_50[['full_name.1', 'activity_per_day_since_acquisition', 'rank_activity_per_day_since_acquisition']]
display_30_best_clients_activity_per_days_entire_period = display_30_best_clients_activity_per_days_entire_period.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_since_acquisition': 'Nombre moyen séance par jours depuis acquisition',
    'rank_activity_per_day_since_acquisition' : 'Classement en fonction de l\'activité depuis l\'acquisition'
    })

#display_30_best_clients_activity_per_days_entire_period
#display(display_30_best_clients_activity_per_days_entire_period.head(30).style.hide(axis='index'))
display(display_30_best_clients_activity_per_days_entire_period.head(30).style.format({"Nombre moyen séance par jours depuis acquisition": "{:.1f}"}).hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_per_days_entire_period.head(30)['Nom du client'], display_30_best_clients_activity_per_days_entire_period.head(30)['Nombre moyen séance par jours depuis acquisition'],  alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres moyen de séances par jours')
plt.title('Classement des 30 premiers clients européens en fonction du nombre moyen de séances par jour depuis l\'acquisition de l\'appareil V4')
_ = plt.xticks(rotation=90)

# sur les 7 derniers jours

In [ ]:
# @title
#display_30_best_clients_activity_per_days_7_days = sorted_activity_europe_50[['full_name.1', 'activity_per_day_7_days', 'rank_activity_per_day_7_days']].sort_values(by=['activity_per_day_7_days'], ascending=False)
display_30_best_clients_activity_per_days_7_days = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 7][['full_name.1', 'activity_per_day_7_days', 'rank_activity_per_day_7_days']].sort_values(by=['activity_per_day_7_days'], ascending=False)
display_30_best_clients_activity_per_days_7_days = display_30_best_clients_activity_per_days_7_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_7_days': 'Nombre moyen séances par jours sur les 7 derniers jours',
    'rank_activity_per_day_7_days' : 'Classement en fonction de l\'activité sur les 7 derniers jours'
    })


#display(display_30_best_clients_activity_per_days_7_days.head(30).style.hide(axis='index'))
display(display_30_best_clients_activity_per_days_7_days.head(30).style.format({"Nombre moyen séances par jours sur les 7 derniers jours": "{:.1f}"}).hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_per_days_7_days.head(30)['Nom du client'], display_30_best_clients_activity_per_days_7_days.head(30)['Nombre moyen séances par jours sur les 7 derniers jours'],  alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres moyen de séances par jours')
plt.title('Classement des 30 premiers clients européens en fonction du nombre moyen de séances par jour sur les 7 derniers jours')
_ = plt.xticks(rotation=90)

# sur les 14 derniers jours

In [ ]:
# @title
#display_30_best_clients_activity_per_days_14_days = sorted_activity_europe_50[['full_name.1', 'activity_per_day_14_days', 'rank_activity_per_day_14_days']].sort_values(by=['activity_per_day_14_days'], ascending=False)
display_30_best_clients_activity_per_days_14_days = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 14][['full_name.1', 'activity_per_day_14_days', 'rank_activity_per_day_14_days']].sort_values(by=['activity_per_day_14_days'], ascending=False)
display_30_best_clients_activity_per_days_14_days = display_30_best_clients_activity_per_days_14_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_14_days': 'Nombre moyen séances par jours sur les 14 derniers jours',
    'rank_activity_per_day_14_days' : 'Classement en fonction de l\'activité sur les 14 derniers jours'
    })


#display(display_30_best_clients_activity_per_days_14_days.head(30).style.hide(axis='index'))
display(display_30_best_clients_activity_per_days_14_days.head(30).style.format({"Nombre moyen séances par jours sur les 14 derniers jours": "{:.1f}"}).hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_per_days_14_days.head(30)['Nom du client'], display_30_best_clients_activity_per_days_14_days.head(30)['Nombre moyen séances par jours sur les 14 derniers jours'],  alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres moyen de séances par jours')
plt.title('Classement des 30 premiers clients européens en fonction du nombre moyen de séances par jour sur les 14 derniers jours')
_ = plt.xticks(rotation=90)

# sur les 30 derniers jours

In [ ]:
# @title
#display_30_best_clients_activity_per_days_30_days = sorted_activity_europe_50[['full_name.1', 'activity_per_day_30_days', 'rank_activity_per_day_30_days']].sort_values(by=['activity_per_day_30_days'], ascending=False)
display_30_best_clients_activity_per_days_30_days = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 30][['full_name.1', 'activity_per_day_30_days', 'rank_activity_per_day_30_days']].sort_values(by=['activity_per_day_30_days'], ascending=False)
display_30_best_clients_activity_per_days_30_days = display_30_best_clients_activity_per_days_30_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_30_days': 'Nombre moyen séances par jours sur les 30 derniers jours',
    'rank_activity_per_day_30_days' : 'Classement en fonction de l\'activité sur les 30 derniers jours'
    })


#display(display_30_best_clients_activity_per_days_30_days.head(30).style.hide(axis='index'))
display(display_30_best_clients_activity_per_days_30_days.head(30).style.format({"Nombre moyen séances par jours sur les 30 derniers jours": "{:.1f}"}).hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_per_days_30_days.head(30)['Nom du client'], display_30_best_clients_activity_per_days_30_days.head(30)['Nombre moyen séances par jours sur les 30 derniers jours'],  alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres moyen de séances par jours')
plt.title('Classement des 30 premiers clients européens en fonction du nombre moyen de séances par jour sur les 30 derniers jours')
_ = plt.xticks(rotation=90)

# sur les 90 derniers jours

In [ ]:
# @title
#display_30_best_clients_activity_per_days_90_days = sorted_activity_europe_50[['full_name.1', 'activity_per_day_90_days', 'rank_activity_per_day_90_days']].sort_values(by=['activity_per_day_90_days'], ascending=False)
display_30_best_clients_activity_per_days_90_days = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 90][['full_name.1', 'activity_per_day_90_days', 'rank_activity_per_day_90_days']].sort_values(by=['activity_per_day_90_days'], ascending=False)
display_30_best_clients_activity_per_days_90_days = display_30_best_clients_activity_per_days_90_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_90_days': 'Nombre moyen séances par jours sur les 90 derniers jours',
    'rank_activity_per_day_90_days' : 'Classement en fonction de l\'activité sur les 90 derniers jours'
    })


#display(display_30_best_clients_activity_per_days_90_days.head(30).style.hide(axis='index'))
display(display_30_best_clients_activity_per_days_90_days.head(30).style.format({"Nombre moyen séances par jours sur les 90 derniers jours": "{:.1f}"}).hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_per_days_90_days.head(30)['Nom du client'], display_30_best_clients_activity_per_days_90_days.head(30)['Nombre moyen séances par jours sur les 90 derniers jours'],  alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres moyen de séances par jours')
plt.title('Classement des 30 premiers clients européens en fonction du nombre moyen de séances par jour sur les 90 derniers jours')
_ = plt.xticks(rotation=90)

# sur 1 an

In [ ]:
# @title
display_30_best_clients_activity_per_days_year = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 365][['full_name.1', 'activity_per_day_year', 'rank_activity_per_day_year']].sort_values(by=['activity_per_day_year'], ascending=False)
display_30_best_clients_activity_per_days_year = display_30_best_clients_activity_per_days_year.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_per_day_year': 'Nombre moyen séances par jours sur 1 an',
    'rank_activity_per_day_year' : 'Classement en fonction de l\'activité sur 1 an'
    })


#display(display_30_best_clients_activity_per_days_year.head(30).style.hide(axis='index'))
display(display_30_best_clients_activity_per_days_year.head(30).style.format({"Nombre moyen séances par jours sur 1 an": "{:.1f}"}).hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_per_days_year.head(30)['Nom du client'], display_30_best_clients_activity_per_days_year.head(30)['Nombre moyen séances par jours sur 1 an'],  alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres moyen de séances par jours')
plt.title('Classement des 30 premiers clients européens en fonction du nombre moyen de séances par jour sur 1 an')
_ = plt.xticks(rotation=90)

# Classement des 30 premiers clients européens en fonction du nombre de séances

# sur 7 jours

In [ ]:
# @title
display_30_best_clients_activity_7_days = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 7][['full_name.1', 'activity_last_7_days', 'rank_activity_per_day_7_days']].sort_values(by=['activity_last_7_days'], ascending=False)
display_30_best_clients_activity_7_days = display_30_best_clients_activity_7_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_last_7_days': 'Nombre de séances sur les 7 derniers jours',
    'rank_activity_per_day_7_days' : 'Classement en fonction de l\'activité sur les 7 derniers jours'
    })


display(display_30_best_clients_activity_7_days.head(30).style.hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_7_days.head(30)['Nom du client'], display_30_best_clients_activity_7_days.head(30)['Nombre de séances sur les 7 derniers jours'], alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres de séances par jours')
plt.title('Classement des 30 clients européens avec le plus de séances sur les 7 derniers jours')
_ = plt.xticks(rotation=90)

# sur 14 jours

In [ ]:
# @title
display_30_best_clients_activity_14_days = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 14][['full_name.1', 'activity_last_14_days', 'rank_activity_per_day_14_days']].sort_values(by=['activity_last_14_days'], ascending=False)
display_30_best_clients_activity_14_days = display_30_best_clients_activity_14_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_last_14_days': 'Nombre de séances sur les 14 derniers jours',
    'rank_activity_per_day_14_days' : 'Classement en fonction de l\'activité sur les 14 derniers jours'
    })


display(display_30_best_clients_activity_14_days.head(30).style.hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_14_days.head(30)['Nom du client'], display_30_best_clients_activity_14_days.head(30)['Nombre de séances sur les 14 derniers jours'], alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres de séances par jours')
plt.title('Classement des 30 clients européens avec le plus de séances sur les 14 derniers jours')
_ = plt.xticks(rotation=90)

# sur 30 jours

In [ ]:
# @title
display_30_best_clients_activity_30_days = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 30][['full_name.1', 'activity_last_30_days', 'rank_activity_per_day_30_days']].sort_values(by=['activity_last_30_days'], ascending=False)
display_30_best_clients_activity_30_days = display_30_best_clients_activity_30_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_last_30_days': 'Nombre de séances sur les 30 derniers jours',
    'rank_activity_per_day_30_days' : 'Classement en fonction de l\'activité sur les 30 derniers jours'
    })


display(display_30_best_clients_activity_30_days.head(30).style.hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_30_days.head(30)['Nom du client'], display_30_best_clients_activity_30_days.head(30)['Nombre de séances sur les 30 derniers jours'], alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres de séances par jours')
plt.title('Classement des 30 clients européens avec le plus de séances sur les 30 derniers jours')
_ = plt.xticks(rotation=90)

# sur 90 jours

In [ ]:
# @title
display_30_best_clients_activity_90_days = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 90][['full_name.1', 'activity_last_90_days', 'rank_activity_per_day_90_days']].sort_values(by=['activity_last_90_days'], ascending=False)
display_30_best_clients_activity_90_days = display_30_best_clients_activity_90_days.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_last_90_days': 'Nombre de séances sur les 90 derniers jours',
    'rank_activity_per_day_90_days' : 'Classement en fonction de l\'activité sur les 90 derniers jours'
    })


display(display_30_best_clients_activity_90_days.head(30).style.hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_90_days.head(30)['Nom du client'], display_30_best_clients_activity_90_days.head(30)['Nombre de séances sur les 90 derniers jours'], alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres de séances par jours')
plt.title('Classement des 30 clients européens avec le plus de séances sur les 90 derniers jours')
_ = plt.xticks(rotation=90)

# sur 1 an

In [ ]:
# @title
display_30_best_clients_activity_year = sorted_activity_europe_50[sorted_activity_europe_50['number_days'] > 365][['full_name.1', 'activity_last_365_days', 'rank_activity_per_day_year']].sort_values(by=['activity_last_365_days'], ascending=False)
display_30_best_clients_activity_year = display_30_best_clients_activity_year.rename(columns={
    'full_name.1' : 'Nom du client',
    'activity_last_365_days': 'Nombre de séances sur 1 an',
    'rank_activity_per_day_year' : 'Classement en fonction de l\'activité sur 1 an'
    })


display(display_30_best_clients_activity_year.head(30).style.hide(axis='index'))

In [ ]:
# @title
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(display_30_best_clients_activity_year.head(30)['Nom du client'], display_30_best_clients_activity_year.head(30)['Nombre de séances sur 1 an'], alpha=0.7)
plt.xlabel('Nom du client')
plt.ylabel('Nombres de séances par jours')
plt.title('Classement des 30 clients européens avec le plus de séances sur 1 an')
_ = plt.xticks(rotation=90)

# Tests sortie détails d'un client

In [ ]:
# @title
# prompt: From the database sorted_activity_europe_50 display the following information:
# 1. The current date
# 2. The rank_activity_per_day of the full_name.1 called T0613 - Aqua Kiné Santé
# 3. The counts of the full_name.1 called T0613 - Aqua Kiné Santé
# 4. The activity_per_day of the full_name.1 called T0613 - Aqua Kiné Santé
# 5. The first_activity of the full_name.1 called T0613 - Aqua Kiné Santé
# 6. The last_activity of the full_name.1 called T0613 - Aqua Kiné Santé
# 7. The number_days of the full_name.1 called T0613 - Aqua Kiné Santé
# 8. The activity_last_7_days of the full_name.1 called T0613 - Aqua Kiné Santé
# 9. The activity_last_14_days of the full_name.1 called T0613 - Aqua Kiné Santé
# 10. The activity_last_30_days of the full_name.1 called T0613 - Aqua Kiné Santé

import datetime

# Assuming 'sorted_activity_europe_50' DataFrame is already available from the previous code

# Filter the DataFrame for the specific full_name.1
target_activity = sorted_activity_europe_50[sorted_activity_europe_50['full_name.1'] == 'T0613 - Aqua Kiné Santé']

if not target_activity.empty:
  current_date = datetime.date.today()
  rank = target_activity['rank_activity_per_day_since_acquisition'].iloc[0]
  counts = target_activity['counts'].iloc[0]
  activity_per_day = target_activity['activity_per_day_since_acquisition'].iloc[0]
  first_activity = target_activity['first_activity'].iloc[0]
  last_activity = target_activity['last_activity'].iloc[0]
  since_asquisition = target_activity['formatted_duration'].iloc[0]
  days_since_acquisition = target_activity['number_days'].iloc[0]
  activity_last_7_days = target_activity['activity_last_7_days'].iloc[0]
  activity_per_day_last_week = target_activity['activity_per_day_7_days'].iloc[0]
  rank_last_week = target_activity['rank_activity_per_day_7_days'].iloc[0]
  activity_last_14_days = target_activity['activity_last_14_days'].iloc[0]
  activity_per_day_last_2_week = target_activity['activity_per_day_14_days'].iloc[0]
  rank_last_2_week = target_activity['rank_activity_per_day_14_days'].iloc[0]
  activity_last_30_days = target_activity['activity_last_30_days'].iloc[0]
  activity_per_day_last_30_days = target_activity['activity_per_day_30_days'].iloc[0]
  rank_last_30_days = target_activity['rank_activity_per_day_30_days'].iloc[0]
  activity_last_90_days = target_activity['activity_last_90_days'].iloc[0]
  activity_per_day_last_90_days = target_activity['activity_per_day_90_days'].iloc[0]
  rank_last_90_days = target_activity['rank_activity_per_day_90_days'].iloc[0]
  activity_last_365_days = target_activity['activity_last_365_days'].iloc[0]
  activity_per_day_last_year = target_activity['activity_per_day_year'].iloc[0]
  rank_last_year = target_activity['rank_activity_per_day_year'].iloc[0]

  print('\033[1;4m' + "Détails pour le client T0613 - Aqua Kiné Santé:" + '\033[0m')
  print("\n")
  print(f"Date d\'aujourd\'hui: {current_date}")
  print(f"Date de la première séance: {first_activity}")
  print(f"Date de la dernière séance: {last_activity}")
  print(f"Première utilisation il y a: {since_asquisition} (ou {days_since_acquisition} jours)")
  print("\n")

  print('\033[1;4m' + "Stats sur les 7 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 7:
    print("Première séance effectué il ya moins de 7 jours.")
  print(f"Nombres de séances: {activity_last_7_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_week}")
  print(f"Classement: {rank_last_week} ")

  print("\n")
  print('\033[1;4m' + "Stats sur les 14 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 14:
    print("Première séance effectué il ya moins de 14 jours.")
  print(f"Nombres de séances: {activity_last_14_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_2_week}")
  print(f"Classement: {rank_last_2_week}")

  print("\n")
  print('\033[1;4m' + "Stats sur les 30 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 30:
    print("Première séance effectué il ya moins de 30 jours.")
  print(f"Nombres de séances: {activity_last_30_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_30_days}")
  print(f"Classement: {rank_last_30_days}")
  print("\n")

  print('\033[1;4m' + "Stats sur les 90 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 90:
    print("Première séance effectué il ya moins de 90 jours.")
  print(f"Nombres de séances: {activity_last_90_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_90_days}")
  print(f"Classement: {rank_last_90_days}")

  print("\n")
  print('\033[1;4m' + "Stats sur 1 an" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 365:
    print("Première séance effectué il ya moins de 365 jours.")
  print(f"Nombres de séances: {activity_last_365_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_year}")
  print(f"Classement: {rank_last_year}")
  print("\n")

  print('\033[1;4m' + "Stats depuis l\'acquisition" + '\033[0m')
  print(f"Nombres de séances: {counts}")
  print(f"Nombres moyen de séances par jour: {activity_per_day}")
  print(f"Classement: {rank}")
else:
  print("T0613 - Aqua Kiné Santé not found in the database.")

In [ ]:
# @title
# prompt: From the database sorted_activity_europe_50 display the following information:
# 1. The current date
# 2. The rank_activity_per_day of the full_name.1 called T0613 - Aqua Kiné Santé
# 3. The counts of the full_name.1 called T0613 - Aqua Kiné Santé
# 4. The activity_per_day of the full_name.1 called T0613 - Aqua Kiné Santé
# 5. The first_activity of the full_name.1 called T0613 - Aqua Kiné Santé
# 6. The last_activity of the full_name.1 called T0613 - Aqua Kiné Santé
# 7. The number_days of the full_name.1 called T0613 - Aqua Kiné Santé
# 8. The activity_last_7_days of the full_name.1 called T0613 - Aqua Kiné Santé
# 9. The activity_last_14_days of the full_name.1 called T0613 - Aqua Kiné Santé
# 10. The activity_last_30_days of the full_name.1 called T0613 - Aqua Kiné Santé

import datetime

# Assuming 'sorted_activity_europe_50' DataFrame is already available from the previous code

# Filter the DataFrame for the specific full_name.1
target_activity = sorted_activity_europe_50[sorted_activity_europe_50['full_name.1'] == 'T0100 - Franck ARCHIMBAUD']

if not target_activity.empty:
  current_date = datetime.date.today()
  rank = target_activity['rank_activity_per_day_since_acquisition'].iloc[0]
  counts = target_activity['counts'].iloc[0]
  activity_per_day = target_activity['activity_per_day_since_acquisition'].iloc[0]
  first_activity = target_activity['first_activity'].iloc[0]
  last_activity = target_activity['last_activity'].iloc[0]
  since_asquisition = target_activity['formatted_duration'].iloc[0]
  days_since_acquisition = target_activity['number_days'].iloc[0]
  activity_last_7_days = target_activity['activity_last_7_days'].iloc[0]
  activity_per_day_last_week = target_activity['activity_per_day_7_days'].iloc[0]
  rank_last_week = target_activity['rank_activity_per_day_7_days'].iloc[0]
  activity_last_14_days = target_activity['activity_last_14_days'].iloc[0]
  activity_per_day_last_2_week = target_activity['activity_per_day_14_days'].iloc[0]
  rank_last_2_week = target_activity['rank_activity_per_day_14_days'].iloc[0]
  activity_last_30_days = target_activity['activity_last_30_days'].iloc[0]
  activity_per_day_last_30_days = target_activity['activity_per_day_30_days'].iloc[0]
  rank_last_30_days = target_activity['rank_activity_per_day_30_days'].iloc[0]
  activity_last_90_days = target_activity['activity_last_90_days'].iloc[0]
  activity_per_day_last_90_days = target_activity['activity_per_day_90_days'].iloc[0]
  rank_last_90_days = target_activity['rank_activity_per_day_90_days'].iloc[0]
  activity_last_365_days = target_activity['activity_last_365_days'].iloc[0]
  activity_per_day_last_year = target_activity['activity_per_day_year'].iloc[0]
  rank_last_year = target_activity['rank_activity_per_day_year'].iloc[0]

  print('\033[1;4m' + "Détails pour le client T0100 - Franck ARCHIMBAUD:" + '\033[0m')
  print("\n")
  print(f"Date d\'aujourd\'hui: {current_date}")
  print(f"Date de la première séance: {first_activity}")
  print(f"Date de la dernière séance: {last_activity}")
  print(f"Première utilisation il y a: {since_asquisition} (ou {days_since_acquisition} jours)")
  print("\n")

  print('\033[1;4m' + "Stats sur les 7 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 7:
    print("Première séance effectué il ya moins de 7 jours.")
  print(f"Nombres de séances: {activity_last_7_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_week}")
  print(f"Classement: {rank_last_week} ")

  print("\n")
  print('\033[1;4m' + "Stats sur les 14 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 14:
    print("Première séance effectué il ya moins de 14 jours.")
  print(f"Nombres de séances: {activity_last_14_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_2_week}")
  print(f"Classement: {rank_last_2_week}")

  print("\n")
  print('\033[1;4m' + "Stats sur les 30 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 30:
    print("Première séance effectué il ya moins de 30 jours.")
  print(f"Nombres de séances: {activity_last_30_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_30_days}")
  print(f"Classement: {rank_last_30_days}")
  print("\n")

  print('\033[1;4m' + "Stats sur les 90 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 90:
    print("Première séance effectué il ya moins de 90 jours.")
  print(f"Nombres de séances: {activity_last_90_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_90_days}")
  print(f"Classement: {rank_last_90_days}")

  print("\n")
  print('\033[1;4m' + "Stats sur 1 an" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 365:
    print("Première séance effectué il ya moins de 365 jours.")
  print(f"Nombres de séances: {activity_last_365_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_year}")
  print(f"Classement: {rank_last_year}")
  print("\n")

  print('\033[1;4m' + "Stats depuis l\'acquisition" + '\033[0m')
  print(f"Nombres de séances: {counts}")
  print(f"Nombres moyen de séances par jour: {activity_per_day}")
  print(f"Classement: {rank}")
else:
  print("T0100 - Franck ARCHIMBAUD not found in the database.")

In [ ]:
# @title
# prompt: From the database sorted_activity_europe_50 display the following information:
# 1. The current date
# 2. The rank_activity_per_day of the full_name.1 called T0613 - Aqua Kiné Santé
# 3. The counts of the full_name.1 called T0613 - Aqua Kiné Santé
# 4. The activity_per_day of the full_name.1 called T0613 - Aqua Kiné Santé
# 5. The first_activity of the full_name.1 called T0613 - Aqua Kiné Santé
# 6. The last_activity of the full_name.1 called T0613 - Aqua Kiné Santé
# 7. The number_days of the full_name.1 called T0613 - Aqua Kiné Santé
# 8. The activity_last_7_days of the full_name.1 called T0613 - Aqua Kiné Santé
# 9. The activity_last_14_days of the full_name.1 called T0613 - Aqua Kiné Santé
# 10. The activity_last_30_days of the full_name.1 called T0613 - Aqua Kiné Santé

import datetime

# Assuming 'sorted_activity_europe_50' DataFrame is already available from the previous code

# Filter the DataFrame for the specific full_name.1
target_activity = sorted_activity_europe_50[sorted_activity_europe_50['full_name.1'] == 'T1680 - Laure GIULIANI']

if not target_activity.empty:
  current_date = datetime.date.today()
  rank = target_activity['rank_activity_per_day_since_acquisition'].iloc[0]
  counts = target_activity['counts'].iloc[0]
  activity_per_day = target_activity['activity_per_day_since_acquisition'].iloc[0]
  first_activity = target_activity['first_activity'].iloc[0]
  last_activity = target_activity['last_activity'].iloc[0]
  since_asquisition = target_activity['formatted_duration'].iloc[0]
  days_since_acquisition = target_activity['number_days'].iloc[0]
  activity_last_7_days = target_activity['activity_last_7_days'].iloc[0]
  activity_per_day_last_week = target_activity['activity_per_day_7_days'].iloc[0]
  rank_last_week = target_activity['rank_activity_per_day_7_days'].iloc[0]
  activity_last_14_days = target_activity['activity_last_14_days'].iloc[0]
  activity_per_day_last_2_week = target_activity['activity_per_day_14_days'].iloc[0]
  rank_last_2_week = target_activity['rank_activity_per_day_14_days'].iloc[0]
  activity_last_30_days = target_activity['activity_last_30_days'].iloc[0]
  activity_per_day_last_30_days = target_activity['activity_per_day_30_days'].iloc[0]
  rank_last_30_days = target_activity['rank_activity_per_day_30_days'].iloc[0]
  activity_last_90_days = target_activity['activity_last_90_days'].iloc[0]
  activity_per_day_last_90_days = target_activity['activity_per_day_90_days'].iloc[0]
  rank_last_90_days = target_activity['rank_activity_per_day_90_days'].iloc[0]
  activity_last_365_days = target_activity['activity_last_365_days'].iloc[0]
  activity_per_day_last_year = target_activity['activity_per_day_year'].iloc[0]
  rank_last_year = target_activity['rank_activity_per_day_year'].iloc[0]

  print('\033[1;4m' + "Détails pour le client T1680 - Laure GIULIANI:" + '\033[0m')
  print("\n")
  print(f"Date d\'aujourd\'hui: {current_date}")
  print(f"Date de la première séance: {first_activity}")
  print(f"Date de la dernière séance: {last_activity}")
  print(f"Première utilisation il y a: {since_asquisition} (ou {days_since_acquisition} jours)")
  print("\n")

  print('\033[1;4m' + "Stats sur les 7 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 7:
    print("Première séance effectué il ya moins de 7 jours.")
  print(f"Nombres de séances: {activity_last_7_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_week}")
  print(f"Classement: {rank_last_week} ")

  print("\n")
  print('\033[1;4m' + "Stats sur les 14 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 14:
    print("Première séance effectué il ya moins de 14 jours.")
  print(f"Nombres de séances: {activity_last_14_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_2_week}")
  print(f"Classement: {rank_last_2_week}")

  print("\n")
  print('\033[1;4m' + "Stats sur les 30 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 30:
    print("Première séance effectué il ya moins de 30 jours.")
  print(f"Nombres de séances: {activity_last_30_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_30_days}")
  print(f"Classement: {rank_last_30_days}")
  print("\n")

  print('\033[1;4m' + "Stats sur les 90 derniers jours" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 90:
    print("Première séance effectué il ya moins de 90 jours.")
  print(f"Nombres de séances: {activity_last_90_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_90_days}")
  print(f"Classement: {rank_last_90_days}")

  print("\n")
  print('\033[1;4m' + "Stats sur 1 an" + '\033[0m')
  if target_activity['number_days'].iloc[0] < 365:
    print("Première séance effectué il ya moins de 365 jours.")
  print(f"Nombres de séances: {activity_last_365_days}")
  print(f"Nombres de séances par jour: {activity_per_day_last_year}")
  print(f"Classement: {rank_last_year}")
  print("\n")

  print('\033[1;4m' + "Stats depuis l\'acquisition" + '\033[0m')
  print(f"Nombres de séances: {counts}")
  print(f"Nombres moyen de séances par jour: {activity_per_day}")
  print(f"Classement: {rank}")
else:
  print("T1680 - Laure GIULIANI not found in the database.")